<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1">Load data</a></span></li><li><span><a href="#Evaluate" data-toc-modified-id="Evaluate-2">Evaluate</a></span></li><li><span><a href="#Cleanup" data-toc-modified-id="Cleanup-3">Cleanup</a></span></li><li><span><a href="#Transform" data-toc-modified-id="Transform-4">Transform</a></span></li></ul></div>

This notebook inputs the Aid Worker Security Database and completes an exploratory analysis.  Data cleanup is also completed and three csv files are created for use in aninteractive visualization.

In [1]:
import pandas as pd 
import numpy as np 
from collections import defaultdict

pd.set_option('Display.max_columns',None)
pd.set_option('Display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')


### Load data

In [2]:
raw_data = pd.read_csv('./data/security_incidents_2021-09-11.csv')
raw_data.head(2)

,Incident ID,Year,Month,Day,Country Code,Country,Region,District,City,UN,INGO,ICRC,NRCS and IFRC,NNGO,Other,Nationals killed,Nationals wounded,Nationals kidnapped,Total nationals,Internationals killed,Internationals wounded,Internationals kidnapped,Total internationals,Total killed,Total wounded,Total kidnapped,Total affected,Gender Male,Gender Female,Gender Unknown,Means of attack,Attack context,Location,Latitude,Longitude,Actor type,Actor name,Details,Verified,Source
0,35,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2.0,0.0,0.0,0,1,0,0,1,0,1,0,1,1,1,0,2,0,0,2,Unknown,Unknown,Unknown,NaN,NaN,Unknown,Unknown,1 ICRC international staff wounded and 1 natio...,Yes,Focal Point
1,1,1997,1.0,NaN,KH,Cambodia,Banteay Meanchey,NaN,NaN,0,0,1.0,0.0,0.0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,Unknown,Unknown,Unknown,14.07092855,103.099916,Unknown,Unknown,1 ICRC national staff killed while working in ...,Archived,Archived


### Evaluate

In [3]:
raw_data.Year.value_counts().sum()

3247

In [4]:
#Analyze missing values across the features
data_columns = []
counts = []
missing_values_count = []

for c in raw_data.columns:
    data_columns.append(c)
    counts.append(raw_data[c].value_counts().sum())
    missing_values_count.append(3247 - raw_data[c].value_counts().sum())
    
data_values_count_df = pd.DataFrame()
data_values_count_df['feature'] = data_columns
data_values_count_df['values_count'] = counts
data_values_count_df['missing_values_count'] = missing_values_count

data_values_count_df = data_values_count_df.sort_values('missing_values_count', ascending=False)
data_values_count_df.head(2)

,feature,values_count,missing_values_count
8,City,2385,862
7,District,2574,673


In [5]:
data_values_count_df.to_csv('./data/data_values_count.csv', index=False)

### Cleanup

In [6]:
#Copy raw data and filter to only include columns needed for the visualization
data = pd.DataFrame()
data = raw_data.copy()

data_columns = [str(x).lower().replace(' ','_') for x in data.columns]
data.columns = data_columns

columns_needed = ['incident_id','year','country','total_killed', 'total_wounded','total_kidnapped','total_affected',
                 'means_of_attack','latitude','longitude']

data = data[columns_needed]

In [7]:
data.head(2)

,incident_id,year,country,total_killed,total_wounded,total_kidnapped,total_affected,means_of_attack,latitude,longitude
0,35,1997,NaN,1,1,0,2,Unknown,NaN,NaN
1,1,1997,Cambodia,1,0,0,1,Unknown,14.07092855,103.099916


Based on the summary below there are only 6 out 3247 records missing the country.  These values will be dropped.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3247 entries, 0 to 3246
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   incident_id      3247 non-null   int64  
 1   year             3247 non-null   int64  
 2   country          3241 non-null   object 
 3   total_killed     3247 non-null   int64  
 4   total_wounded    3247 non-null   int64  
 5   total_kidnapped  3247 non-null   int64  
 6   total_affected   3247 non-null   int64  
 7   means_of_attack  3247 non-null   object 
 8   latitude         3241 non-null   object 
 9   longitude        3241 non-null   float64
dtypes: float64(1), int64(6), object(3)
memory usage: 253.8+ KB


In [9]:
#Remove attacks missing a country
data = data.dropna(how='any',axis=0)
data = data[data['year'] < 2021]

#one row has a bad value of a comma with the latitude, so the comma is removed
data['latitude'] = [pd.to_numeric(str(x).replace(',','')) for x in data.latitude.values]

### Transform

In [10]:
#Create means_of_attack_condensed feature.  There are 10 methods of attack, that I'm condensing into broader categories
#of kidnapping, shooting, explosives, bodily assualt, and Unknown.  This will make the dashboard easier to understand.

def am_condensed(row):
    am = row['means_of_attack'] 
    
    if 'Kidnap' in am:
        out_str = 'Kidnapping'
        
    elif am == 'Shooting':
        out_str = 'Shooting'
    
    elif am == 'Bodily assault':
        out_str = 'Bodily assault'
        
    elif am == 'Unknown':
        out_str = 'Unknown'
    
    else:
        out_str = 'Explosives'
    
    return out_str

data['means_of_attack_condensed'] = data.apply(am_condensed, axis=1)

In [11]:
#Add a risk score for each country for each year: formula I derived for risk is a multiplier of 
#kidnapped = 1, wounded = 2, killed = 3
data['risk'] = data.total_kidnapped + (data.total_wounded * 2) + (data.total_killed * 3)

In [12]:
#Create a risk score for each country for all years combined and store in a dataframe
country_risk_score_no_year = data.copy()

country_risk_score_no_year = country_risk_score_no_year[['country', 'total_killed', 'total_wounded', 'total_kidnapped', 
                                        'total_affected','risk']]

country_risk_score_no_year = country_risk_score_no_year.groupby(['country']).sum()
country_risk_score_no_year = country_risk_score_no_year.reset_index()

country_risk_score_no_year.head(2)

,country,total_killed,total_wounded,total_kidnapped,total_affected,risk
0,Afghanistan,454,354,464,1272,2534
1,Algeria,8,4,0,12,32


In [13]:
#Create a dictionary of each country's overall risk score
country = country_risk_score_no_year.country
risk = country_risk_score_no_year.risk

ors_dict = dict(zip(country,risk))

In [14]:
#Add each country's risk score for all_years as a feature
data['risk_allyears'] = [ors_dict[x[2]] for x in data.values]

In [15]:
#Output clean file to csv
data.to_csv('./data/security_incidents_clean_data.csv', index=False)